# The ACME Dataset <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-the-data" data-toc-modified-id="Getting-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting the data</a></span></li><li><span><a href="#The-data" data-toc-modified-id="The-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The data</a></span></li><li><span><a href="#The-metadata" data-toc-modified-id="The-metadata-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The metadata</a></span></li><li><span><a href="#Details-to-note-and-summary-statistics" data-toc-modified-id="Details-to-note-and-summary-statistics-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Details to note and summary statistics</a></span><ul class="toc-item"><li><span><a href="#Initial-onset-time" data-toc-modified-id="Initial-onset-time-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Initial onset time</a></span></li><li><span><a href="#Excerpt-lengths" data-toc-modified-id="Excerpt-lengths-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Excerpt lengths</a></span></li></ul></li><li><span><a href="#Compressed-data-for-modelling" data-toc-modified-id="Compressed-data-for-modelling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compressed data for modelling</a></span></li></ul></div>

In [ ]:
import logging
from pathlib import Path

import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
from tqdm import tqdm

import mdtk
from mdtk.fileio import COLNAMES, csv_to_df
from utils import download_file, unzip_file, plot_from_df

LOGGER = logging.getLogger()

# Getting the data

First, let's obtain the acme dataset, and unzip it adjacent to this file.

In [ ]:
url = f"https://github.com/JamesOwers/ACME/raw/master/ACMEv1.0.zip"
download_file(url)
unzip_file("ACMEv1.0.zip")

The dataset should now exist at a directory called `ACMEv1.0/acme`, adjacent to this notebook. The contents are as follows:

* `clean` - contains the extracted clean excerpts as csv files
* `altered` - contains the excerpts as csv files altered by the degradations described in `metadata.csv`

`metadata.csv` describes:

* (the id number for) the type of degradation used for the alteration
* the path for the altered and clean files
* which split (train, valid, test) the file should be used in
* in which corpus csv and on what line the file is located

`degradation_ids.csv` is a mapping of degradation name to the id number used in `metadata.csv`

The `{train,valid,test}_cmd_corpus.csv` are command-based (note_on, note_off, shift) versions of the acme data more convenient for our provided pytorch Dataset classes avaialble in the [MIDI Degradation Toolkit](https://github.com/JamesOwers/midi_degradation_toolkit).

Similarly, `{train,valid,test}_pr_corpus.csv` are piano-roll-based versions of the acme data more convenient for our provided pytorch Dataset classes avaialble in the [MIDI Degradation Toolkit](https://github.com/JamesOwers/midi_degradation_toolkit).

In [ ]:
acme_root = Path("ACMEv1.0", "acme")

# The data

First let's look at the way the excerpts are stored as csvs. We call each of these a Note DataFrame, or a `note_df`, and they are an ordered list of the notes which have occurred in the excerpt.

Here's a random clean excerpt:

In [ ]:
random.seed(1337)
rand_filepath = random.choice(list(Path(acme_root).glob("clean/**/*.csv")))
rand_filepath

The columns are:
1. onset time of the note in ms
2. the track used to play the note (read: instrument)
3. the MIDI pitch number (60 is C4)
4. the duration of the note in ms

In [ ]:
df = pd.read_csv(rand_filepath, names=["onset", "track", "pitch", "dur"])
with pd.option_context("display.max_rows", 10):
    display(df)

In [ ]:
plot_from_df(df)

Each excerpt has a matching degraded version (excluding those which are not degraded!)

In [ ]:
alt_path = Path(*rand_filepath.parts[:-3], "altered", *rand_filepath.parts[-2:])
alt_path

In [ ]:
alt_df = pd.read_csv(alt_path, names=["onset", "track", "pitch", "dur"])
with pd.option_context("display.max_rows", 10):
    display(alt_df)
plot_from_df(alt_df)

Can you see where it has been altered?! There is only one degradation applied to each altered example.

# The metadata

All the information about how the excerpts have been degraded is contained within `metadata.csv`. The names of the degradations are in `degredation_ids.csv`. Each row of `metadata.csv` gives the following information for each excerpt:
* the clean and altered path (relative to the root directory),
* a flag showing if the excerpt was degraded, and which degradation was used,
* and the location of the data within the compressed corpus files (more on these later).

These can be used by a data parser to get the required input data, labels, or target data required.


In [ ]:
meta_df = pd.read_csv(Path(acme_root, "metadata.csv"))
display(meta_df.dtypes)
meta_df.head()

In [ ]:
deg_df = pd.read_csv(Path(acme_root, "degradation_ids.csv"), index_col="id")
deg_df

For example, taking a few random rows from the metadata:

In [ ]:
random.seed(1337)
idx_sample = random.sample(range(len(meta_df)), 4)
idx_sample

In [ ]:
def inspect_by_index(idx, meta_df):
    excerpt_meta = meta_df.loc[idx]
    deg_name = deg_df.loc[excerpt_meta.degradation_id, "degradation_name"]
    clean_path = acme_root / Path(excerpt_meta["clean_csv_path"])
    alt_path = acme_root / Path(excerpt_meta["altered_csv_path"])
    clean_df = csv_to_df(clean_path)
    alt_df = csv_to_df(alt_path)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)
    plot_from_df(clean_df, ax=ax0)
    ax0.set_title("clean")
    plot_from_df(alt_df, ax=ax1)
    ax1.set_title("altered")
    title = f"Degradataion: {deg_name}, excerpt index: {idx}"
    plt.suptitle(title)
    plt.tight_layout()


for idx in idx_sample:
    inspect_by_index(idx, meta_df)

# Details to note and summary statistics

Most details are included in the paper in section 3. But we outline details here too.

* all excerpts are approximately 5 seconds long

* the onset time for the excerpt doesn't match the file it was taken from,
* but the onset times of the clean and altered examples are aligned
* a random amount of silence was added to the start of each excerpt to allow for the first note to be degraded
* an equal proportion of all the degradations

In [ ]:
all_clean_df = [
    csv_to_df(acme_root / Path(path)) for path in tqdm(meta_df.clean_csv_path)
]
# N.B. this includes clean dfs if deg is None
all_alt_df = [
    csv_to_df(acme_root / Path(path)) for path in tqdm(meta_df.altered_csv_path)
]

## Initial onset time

The inital onsets are uniformly distributed between 0 and 200 ms

In [ ]:
clean_first_onsets = [df.at[0, "onset"] for df in tqdm(all_clean_df)]
alt_first_onsets = [df.at[0, "onset"] for df in tqdm(all_alt_df)]

In [ ]:
sns.distplot(clean_first_onsets)
plt.title(f"min: {min(clean_first_onsets)}, max: {max(clean_first_onsets)}")
plt.xlabel("first onset time (ms)")
plt.show()

For each excerpt, if we observe the difference between the first onsets, we see they are normally the same, but that some degradations alter the first note in some way, causing a difference.

In [ ]:
diff_first_onsets = [
    alt - clean for alt, clean in zip(alt_first_onsets, clean_first_onsets)
]

In [ ]:
sns.countplot([diff == 0 for diff in diff_first_onsets])
plt.xlabel("Clean and altered excerpts have the same first onset")
plt.show()

In [ ]:
sns.distplot(diff_first_onsets)
plt.title(f"min: {min(diff_first_onsets)}, max: {max(diff_first_onsets)}")
plt.xlabel("alt first onset - clean first onset")
plt.show()

In [ ]:
nonzero_diffs = [diff for diff in diff_first_onsets if diff != 0]
print(
    f"There are {len(nonzero_diffs)} examples where the onset time of the "
    "clean and altered excerpt are not the same"
)
sns.distplot(nonzero_diffs)
plt.title(f"min: {min(nonzero_diffs)}, max: {max(nonzero_diffs)}")
plt.xlabel("alt first onset - clean first onset (non-zero only)")
plt.show()

Let's have a look at the most extreme examples where then onset of the clean and altered versions differ the most.

In [ ]:
def argmax(sequence):
    _, idx = max((val, idx) for idx, val in enumerate(sequence))
    return idx


def argmin(sequence):
    _, idx = min((val, idx) for idx, val in enumerate(sequence))
    return idx

In [ ]:
inspect_by_index(argmax(diff_first_onsets), meta_df)

In [ ]:
inspect_by_index(argmin(diff_first_onsets), meta_df)

In [ ]:
nonzero_diff_idx = [idx for idx, diff in enumerate(diff_first_onsets) if diff != 0]

In [ ]:
meta_df["degradation_str"] = meta_df.degradation_id.apply(lambda x: deg_df.loc[x])

In [ ]:
sns.countplot(meta_df.loc[nonzero_diff_idx, "degradation_str"])
plt.title("Degradation types which cause nonzero onset differences")
plt.show()

## Excerpt lengths

The code for selecting excerpts from a whole piece is `mdtk.df_utils.get_random_excerpt`.

By default, and as used for ACMEv1.0, we select an  ~5 second excerpt from each piece by choosing a random note and all notes beginning in the subsequent 5 seconds, but require that at least 10 notes be present. The excerpt ends when the last held note is released.

As such, if there are notes held for a long time, or if there are relatively few notes within the selected 5 second window, the excerpt may be longer or shorter than 5 seconds respectively.

In [ ]:
?mdtk.df_utils.get_random_excerpt

In [ ]:
clean_last_offsets = [(df["onset"] + df["dur"]).max() for df in tqdm(all_clean_df)]
alt_last_offsets = [(df["onset"] + df["dur"]).max() for df in tqdm(all_alt_df)]

In [ ]:
clean_dur = [
    last_off - first_on for first_on, last_off in 
    zip(clean_first_onsets, clean_last_offsets)
]

In [ ]:
sns.distplot(clean_dur)
plt.title(f"min: {min(clean_dur)}, max: {max(clean_dur)}")
plt.xlabel("excerpt duration (ms)")
plt.show()

In [ ]:
inspect_by_index(argmax(clean_dur), meta_df)

In [ ]:
inspect_by_index(argmin(clean_dur), meta_df)

# Compressed data for modelling

We have also provided data in two other formats - piano roll and command format. We detail these in a subsequent notebook: [04_data_parsers_and_degrader.ipynb](./04_data_parsers_and_degrader.ipynb).